In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/friedman/many"
results_dir_relu = "results/regression/single_layer/relu/friedman"
results_dir_tanh = "results/regression/single_layer/tanh/friedman"

model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe"]#, "Dirichlet Student T"]
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh"]#, "Dirichlet Student T tanh"]


relu_fits = {}
tanh_fits = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu,
        include_prior=False,
    )
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    

    relu_fits[base_config_name] = relu_fit  # use clean key
    tanh_fits[base_config_name] = tanh_fit  # use clean key
    


In [3]:
data_dir = f"datasets/friedman"
results_dir_relu = "results/regression/single_layer/relu/friedman"
results_dir_tanh = "results/regression/single_layer/tanh/friedman"


files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu,
        include_prior=False,
    )
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    

    relu_fits[base_config_name] = relu_fit  # use clean key
    tanh_fits[base_config_name] = tanh_fit  # use clean key
    


In [ ]:
data_dir = f"datasets/friedman_correlated/many"
results_dir_relu_correlated = "results/regression/single_layer/relu/friedman_correlated"
results_dir_tanh_correlated = "results/regression/single_layer/tanh/friedman_correlated"

relu_fits_correlated = {}
tanh_fits_correlated = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit_correlated = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu_correlated,
        models=model_names_relu,
        include_prior=False,
    )
    
    tanh_fit_correlated = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh_correlated,
        models=model_names_tanh,
        include_prior=False,
    )
    

    relu_fits_correlated[base_config_name] = relu_fit_correlated  # use clean key
    tanh_fits_correlated[base_config_name] = tanh_fit_correlated  # use clean key


In [ ]:
data_dir = f"datasets/friedman_correlated"
results_dir_relu = "results/regression/single_layer/relu/friedman_correlated"
results_dir_tanh = "results/regression/single_layer/tanh/friedman_correlated"

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit_correlated = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu,
        include_prior=False,
    )
    
    tanh_fit_correlated = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    

    relu_fits_correlated[base_config_name] = relu_fit_correlated  # use clean key
    tanh_fits_correlated[base_config_name] = tanh_fit_correlated  # use clean key
    


In [5]:
import re
import numpy as np
import pandas as pd
from properscoring import crps_ensemble

_FRIEDMAN_KEY = re.compile(r"Friedman_N(\d+)_p\d+_sigma([\d.]+)_seed(\d+)")

def extract_friedman_metadata(key: str):
    """
    Parse 'Friedman_N{N}_p10_sigma{sigma}_seed{seed}' -> (N:int, sigma:float, seed:int)
    Returns (None, None, None) if it doesn't match.
    """
    m = _FRIEDMAN_KEY.search(key)
    if not m:
        return None, None, None
    N = int(m.group(1))
    sigma = float(m.group(2))
    seed = int(m.group(3))
    return N, sigma, seed


In [6]:
def compute_rmse_from_fits(all_fits, model_names=None, folder="friedman"):
    """
    Iterate over all dataset keys in `all_fits` (e.g., relu_fits or tanh_fits).
    For each model in `model_names` (or all models found if None), compute:
      - RMSE for each posterior draw
      - RMSE of the posterior mean predictor

    Returns:
        df_rmse: long DF with one row per posterior draw.
        df_posterior_rmse: one row per model/dataset with posterior-mean RMSE.
    """
    rmse_rows = []
    post_mean_rows = []

    for dataset_key, model_dict in all_fits.items():
        N, sigma, seed = extract_friedman_metadata(dataset_key)
        if N is None:
            # Skip non-Friedman entries if any
            continue

        path = f"datasets/{folder}/Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}.npz"
        try:
            data = np.load(path)
            y_test = data["y_test"].squeeze()  # shape (N_test,)
        except FileNotFoundError:
            path = f"datasets/friedman/many/Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}.npz"
            data = np.load(path)
            y_test = data["y_test"].squeeze()  # shape (N_test,)
            #print(f"[SKIP] y_test not found: {path}")
            #continue

        # Choose which models to evaluate
        models_to_eval = model_names or list(model_dict.keys())

        for model in models_to_eval:
            # Some entries may be missing
            entry = model_dict.get(model, None)
            if not entry or "posterior" not in entry:
                print(f"[SKIP] Missing posterior: {dataset_key} -> {model}")
                continue

            fit = entry["posterior"]

            # Expecting (S, N_test, 1) or (S, N_test)
            output_test = fit.stan_variable("output_test")
            if output_test.ndim == 3 and output_test.shape[-1] == 1:
                preds = output_test[..., 0]  # (S, N_test)
            elif output_test.ndim == 2:
                preds = output_test  # (S, N_test)
            else:
                raise ValueError(f"Unexpected output_test shape {output_test.shape} for {dataset_key} -> {model}")

            # Per-sample RMSE
            sq_err = (preds - y_test[None, :])**2  # (S, N_test)
            rmse_per_sample = np.sqrt(np.mean(sq_err, axis=1))  # (S,)

            for s_idx, rmse in enumerate(rmse_per_sample):
                rmse_rows.append({
                    "dataset_key": dataset_key,
                    "model": model,
                    "N": N,
                    "sigma": sigma,
                    "seed": seed,
                    "sample_idx": s_idx,
                    "rmse": float(rmse)
                })

            # Posterior-mean RMSE
            posterior_mean = preds.mean(axis=0)  # (N_test,)
            post_mean_rmse = float(np.sqrt(np.mean((posterior_mean - y_test)**2)))
            post_mean_rows.append({
                "dataset_key": dataset_key,
                "model": model,
                "N": N,
                "sigma": sigma,
                "seed": seed,
                "posterior_mean_rmse": post_mean_rmse
            })

    df_rmse = pd.DataFrame(rmse_rows)
    df_posterior_rmse = pd.DataFrame(post_mean_rows)
    return df_rmse, df_posterior_rmse


def compute_crps_from_fits(all_fits, model_names=None):
    """
    Compute CRPS per dataset/model using all posterior predictive samples.

    Returns:
        df_crps: one row per dataset/model with mean CRPS.
    """
    rows = []

    for dataset_key, model_dict in all_fits.items():
        N, sigma, seed = extract_friedman_metadata(dataset_key)
        if N is None:
            continue

        path = f"datasets/friedman/Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}.npz"
        try:
            data = np.load(path)
            y_test = data["y_test"].squeeze()  # (N_test,)
        except FileNotFoundError:
            print(f"[SKIP] y_test not found: {path}")
            continue

        models_to_eval = model_names or list(model_dict.keys())

        for model in models_to_eval:
            entry = model_dict.get(model, None)
            if not entry or "posterior" not in entry:
                print(f"[SKIP] Missing posterior: {dataset_key} -> {model}")
                continue

            fit = entry["posterior"]
            output_test = fit.stan_variable("output_test")

            # Expecting (S, N_test, 1) or (S, N_test)
            if output_test.ndim == 3 and output_test.shape[-1] == 1:
                preds = output_test[..., 0]  # (S, N_test)
            elif output_test.ndim == 2:
                preds = output_test  # (S, N_test)
            else:
                raise ValueError(f"Unexpected output_test shape {output_test.shape} for {dataset_key} -> {model}")

            # crps_ensemble expects shape (N_test, S)
            crps_point = crps_ensemble(y_test, preds.T)  # (N_test,)
            rows.append({
                "dataset_key": dataset_key,
                "model": model,
                "N": N,
                "sigma": sigma,
                "seed": seed,
                "crps": float(crps_point.mean())
            })

    df_crps = pd.DataFrame(rows)
    return df_crps


In [ ]:
# Evaluate ReLU models
df_rmse_relu, df_posterior_rmse_relu = compute_rmse_from_fits(
    relu_fits, model_names_relu  # or None to use all found
)
# df_crps_relu = compute_crps_from_fits(
#     relu_fits, model_names_relu
# )

df_rmse_relu_correlated, df_posterior_rmse_relu_correlated = compute_rmse_from_fits(
    relu_fits_correlated, model_names_relu, folder = "friedman_correlated/many"
)

# # Evaluate tanh models
df_rmse_tanh, df_posterior_rmse_tanh = compute_rmse_from_fits(
    tanh_fits, model_names_tanh
)
# df_crps_tanh = compute_crps_from_fits(
#     tanh_fits, model_names_tanh
# )

df_rmse_tanh_correlated, df_posterior_rmse_tanh_correlated = compute_rmse_from_fits(
    tanh_fits_correlated, model_names_tanh, folder = "friedman_correlated"
)


In [18]:
summary_relu = df_rmse_relu.groupby(["model", "N"]).agg(
    acc_mean=("rmse", "mean"),
    acc_std=("rmse", "std"),
    #nll_mean=("nll", "mean"),
    #nll_std=("nll", "std"),
).reset_index()

#print(summary)


In [19]:
summary_posterior_relu = df_posterior_rmse_relu.groupby(["model", "N"]).agg(
    acc_mean=("posterior_mean_rmse", "mean"),
    acc_std=("posterior_mean_rmse", "std"),
    #nll_mean=("nll", "mean"),
    #nll_std=("nll", "std"),
).reset_index()


In [ ]:
summary_relu_correlated = df_rmse_relu_correlated.groupby(["model", "N"]).agg(
    acc_mean=("rmse", "mean"),
    acc_std=("rmse", "std"),
    #nll_mean=("nll", "mean"),
    #nll_std=("nll", "std"),
).reset_index()

print(summary_relu_correlated)


In [21]:
summary_posterior_relu_correlated = df_posterior_rmse_relu_correlated.groupby(["model", "N"]).agg(
    acc_mean=("posterior_mean_rmse", "mean"),
    acc_std=("posterior_mean_rmse", "std"),
    #nll_mean=("nll", "mean"),
    #nll_std=("nll", "std"),
).reset_index()


In [22]:
summary_tanh = df_rmse_tanh.groupby(["model", "N"]).agg(
    acc_mean=("rmse", "mean"),
    acc_std=("rmse", "std"),
    #nll_mean=("nll", "mean"),
    #nll_std=("nll", "std"),
).reset_index()

In [23]:
summary_posterior_tanh = df_posterior_rmse_tanh.groupby(["model", "N"]).agg(
    acc_mean=("posterior_mean_rmse", "mean"),
    acc_std=("posterior_mean_rmse", "std"),
    #nll_mean=("nll", "mean"),
    #nll_std=("nll", "std"),
).reset_index()

In [24]:
summary_tanh_correlated = df_rmse_tanh_correlated.groupby(["model", "N"]).agg(
    acc_mean=("rmse", "mean"),
    acc_std=("rmse", "std"),
    #nll_mean=("nll", "mean"),
    #nll_std=("nll", "std"),
).reset_index()

In [25]:
summary_posterior_tanh_correlated = df_posterior_rmse_tanh_correlated.groupby(["model", "N"]).agg(
    acc_mean=("posterior_mean_rmse", "mean"),
    acc_std=("posterior_mean_rmse", "std"),
    #nll_mean=("nll", "mean"),
    #nll_std=("nll", "std"),
).reset_index()

In [ ]:
print(summary_relu.to_latex(index=False, float_format="%.3f"))

print(summary_tanh.to_latex(index=False, float_format="%.3f"))

print(summary_tanh_correlated.to_latex(index=False, float_format="%.3f"))

In [ ]:
print(summary_posterior_relu.to_latex(index=False, float_format="%.3f"))

print(summary_posterior_tanh.to_latex(index=False, float_format="%.3f"))

print(summary_posterior_tanh_correlated.to_latex(index=False, float_format="%.3f"))

In [ ]:
sns.pointplot(data=df_rmse_relu
, x="N", y="rmse", hue="model", dodge=True, errorbar="sd")
plt.title("RMSE relu")
plt.show()

sns.pointplot(data=df_rmse_tanh, x="N", y="rmse", hue="model", dodge=True, errorbar="sd")
plt.title("RMSE tanh")
plt.show()


In [ ]:
sns.pointplot(data=df_rmse_relu_correlated
, x="N", y="rmse", hue="model", dodge=True, errorbar="sd")
plt.title("RMSE relu")
plt.show()

sns.pointplot(data=df_rmse_tanh_correlated, x="N", y="rmse", hue="model", dodge=True, errorbar="sd")
plt.title("RMSE tanh")
plt.show()

In [ ]:
sns.pointplot(data=df_posterior_rmse_relu, x="N", y="posterior_mean_rmse", hue="model", dodge=True, errorbar="sd")
plt.title("RMSE relu")
plt.show()

sns.pointplot(data=df_posterior_rmse_tanh, x="N", y="posterior_mean_rmse", hue="model", dodge=True, errorbar="sd")
plt.title("RMSE tanh")
plt.show()


In [ ]:
sns.pointplot(data=df_posterior_rmse_relu_correlated, x="N", y="posterior_mean_rmse", hue="model", dodge=True, errorbar="sd")
plt.title("RMSE relu")
plt.show()

sns.pointplot(data=df_posterior_rmse_tanh_correlated, x="N", y="posterior_mean_rmse", hue="model", dodge=True, errorbar="sd")
plt.title("RMSE tanh")
plt.show()
